In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# import sys

# # Add the folder containing your python scripts to the path
# sys.path.append('/content/drive/MyDrive/cukh03')

In [1]:
!pip install kagglehub
import kagglehub

# Download latest version
path = kagglehub.dataset_download("priyanagda/cuhk03")

print("Path to dataset files:", path)

/opt/anaconda3/envs/gen-ai/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 2.69G/2.69G [02:52<00:00, 16.7MB/s]

Extracting files...


Path to dataset files: /Users/misanmeggison/.cache/kagglehub/datasets/priyanagda/cuhk03/versions/3


In [3]:
import os
import json

proj_dir = '/Users/misanmeggison/Downloads/cukh03'
data_dir = os.path.join(proj_dir, 'cuhk03')

def prepare_data_list(data_list_path, save_dir=proj_dir):
    os.makedirs(save_dir, exist_ok=True)
    with open(data_list_path, 'r') as f:
        data = json.load(f)
        print(data)
        print(len(data))
    image_list = []

    for item in data[0].get('train', []):
        image_path = item[0]
        image_path = image_path.replace('\\', '/')
        image_path = image_path.split('/')[-1]  # Get the filename only
        image_list.append(image_path)

    output_file = os.path.join(save_dir, 'train.txt')
    with open(output_file, 'w') as f:
        for img in image_list:
            f.write(f"{img}\n")

    print(f"Data list saved to {output_file}")

    # Process 'query' as 'test'
    query_list = []
    for item in data[0].get('query', []):
        image_path = item[0].replace('\\', '/')
        image_path = image_path.split('/')[-1]
        query_list.append(image_path)

    test_file = os.path.join(save_dir, 'test.txt')
    with open(test_file, 'w') as f:
        for img in query_list:
            f.write(f"{img}\n")
    print(f"[✓] Saved test.txt to: {test_file}")

    return os.path.join(save_dir, 'train.txt')

In [4]:
# prepare_data_list(os.path.join(data_dir, 'cuhk03_new_protocol_new_labels.json'))

In [5]:
import torch
import numpy as np
import json
from torchvision import datasets, transforms
from torch.utils.data.sampler import SubsetRandomSampler
from PIL import Image
from random import choice, sample
from torch.utils.data import  Dataset,  ConcatDataset
import random

In [6]:
import os
from PIL import Image
from torchvision import transforms

def reid_data_prepare(data_list_path, train_dir_path):
    """
    Prepares Re-ID data by loading images, transforming them, and organizing them by class.

    This updated version skips any image files listed in the data_list_path that
    are not actually present in the train_dir_path.
    """
    class_img_labels = dict()
    class_cnt = -1
    last_label = -2

    h, w = 224, 224

    # Define the image transformations
    transform_train_list = [
        transforms.Resize((h, w), interpolation=transforms.InterpolationMode.BICUBIC),
        transforms.ToTensor()
    ]
    transform = transforms.Compose(transform_train_list)

    # Open the file containing the list of image paths
    with open(data_list_path, 'r') as f:
        for line in f:
            line = line.strip()
            img_filename = line

            # Determine the label based on the dataset name in the path
            if "cuhk01" in data_list_path:
                lbl = int(img_filename[:4])
            elif "cuhk03" in data_list_path:
                lbl = int(img_filename.split('_')[1])
            else:
                lbl = int(img_filename.split('_')[0])

            # Update class counter and dictionary for new labels
            if lbl != last_label:
                class_cnt += 1
                class_img_labels[str(class_cnt)] = []
            last_label = lbl

            # --- Start of updated block ---
            try:
                # Construct the full image path
                full_img_path = os.path.join(train_dir_path, img_filename)

                # Attempt to open the image
                img = Image.open(full_img_path)

                # If successful, transform and append the image
                img = transform(img)
                class_img_labels[str(class_cnt)].append(img)

                print(f"Loaded and transformed image: {full_img_path}")

            except FileNotFoundError:
                # If the file does not exist, print a warning (optional) and skip to the next image
                print(f"Warning: File not found at {full_img_path}. Skipping.")
                continue
            # --- End of updated block ---

    return class_img_labels

In [7]:
import os
from PIL import Image
from torchvision import transforms

def reid_data_prepare(data_list_path, train_dir_path):
    """
    Prepares Re-ID data by loading images, transforming them, and organizing them by class.

    This updated version skips any image files listed in the data_list_path that
    are not actually present in the train_dir_path.
    """
    class_img_labels = dict()
    class_cnt = -1
    last_label = -2

    h, w = 224, 224

    # Define the image transformations
    transform_train_list = [
        transforms.Resize((h, w), interpolation=transforms.InterpolationMode.BICUBIC),
        transforms.ToTensor()
    ]
    transform = transforms.Compose(transform_train_list)

    # Open the file containing the list of image paths
    with open(data_list_path, 'r') as f:
        for line in f:
            line = line.strip()
            img_filename = line

            # Determine the label based on the dataset name in the path
            if "cuhk01" in data_list_path:
                lbl = int(img_filename[:4])
            elif "cuhk03" in data_list_path:
                lbl = int(img_filename.split('_')[1])
            else:
                lbl = int(img_filename.split('_')[0])

            # Update class counter and dictionary for new labels
            if lbl != last_label:
                class_cnt += 1
                class_img_labels[str(class_cnt)] = []
            last_label = lbl

            # --- Start of updated block ---
            try:
                # Construct the full image path
                full_img_path = os.path.join(train_dir_path, img_filename)

                # Attempt to open the image
                img = Image.open(full_img_path)

                # If successful, transform and append the image
                img = transform(img)
                class_img_labels[str(class_cnt)].append(img)

                print(f"Loaded and transformed image: {full_img_path}")

            except FileNotFoundError:
                # If the file does not exist, print a warning (optional) and skip to the next image
                print(f"Warning: File not found at {full_img_path}. Skipping.")
                continue
            # --- End of updated block ---

    return class_img_labels

In [8]:
# !pip install numpy
class_image_labels = reid_data_prepare(os.path.join(data_dir, 'train.txt'), os.path.join(data_dir, 'images_labeled'))

Loaded and transformed image: /Users/misanmeggison/Downloads/cukh03/cuhk03/images_labeled/2_186_1_01.png
Loaded and transformed image: /Users/misanmeggison/Downloads/cukh03/cuhk03/images_labeled/2_186_1_02.png
Loaded and transformed image: /Users/misanmeggison/Downloads/cukh03/cuhk03/images_labeled/2_186_1_03.png
Loaded and transformed image: /Users/misanmeggison/Downloads/cukh03/cuhk03/images_labeled/2_186_1_04.png
Loaded and transformed image: /Users/misanmeggison/Downloads/cukh03/cuhk03/images_labeled/2_186_1_05.png
Loaded and transformed image: /Users/misanmeggison/Downloads/cukh03/cuhk03/images_labeled/2_186_2_06.png
Loaded and transformed image: /Users/misanmeggison/Downloads/cukh03/cuhk03/images_labeled/2_186_2_07.png
Loaded and transformed image: /Users/misanmeggison/Downloads/cukh03/cuhk03/images_labeled/2_186_2_08.png
Loaded and transformed image: /Users/misanmeggison/Downloads/cukh03/cuhk03/images_labeled/2_186_2_09.png
Loaded and transformed image: /Users/misanmeggison/Down

In [9]:
# get count of unique key values
# class_image_labels

### N-tuple Loss (from Eq.(6))

Given an anchor sample $ \mathbf{x}_a $, a positive sample $ \mathbf{x}^+ $, and $ N - 2 $ negative samples $ \mathbf{x}_k^-,\ k=1,\dots,N-2 $, the N-tuple loss is defined as:

$$
\mathcal{L}_{\text{N-tuple}} = -\log \left(
\frac{
\exp\left( \frac{1}{\tau} S(\mathbf{x}_a, \mathbf{x}^+) \right)
}{
\exp\left( \frac{1}{\tau} S(\mathbf{x}_a, \mathbf{x}^+) \right)
+ \sum_{k=1}^{N-2} \exp\left( \frac{1}{\tau} S(\mathbf{x}_a, \mathbf{x}_k^-) \right)
}
\right)
$$

Where:
- $ S(\cdot, \cdot) $ is a similarity function (e.g., cosine similarity)
- $ \tau $ is a temperature scaling factor



In [10]:
from random import choice, sample
import torch

def ntuple_reid_data(class_img_labels, class_list, N=5, samples_per_class=5):
    """
    Create data for N-tuple loss:
    Each sample = (anchor, positive, N-2 negatives from different classes)

    Returns:
        anchors:  Tensor (B, C, H, W)
        positives: Tensor (B, C, H, W)
        negatives: Tensor (B, N-2, C, H, W)
    """
    anchors = []
    positives = []
    negatives = []

    if not class_img_labels:
        # Handle case where the entire input is empty
        return torch.empty(0), torch.empty(0), torch.empty(0)

    assert N >= 3, "N must be at least 3 (anchor + positive + 1 neg)"

    # --- Start of Fix ---
    # 1. Pre-filter the class list to get lists of valid classes for anchors and negatives.
    # Anchors/positives require at least 2 images per class.
    valid_anchor_classes = [cls for cls in class_list if class_img_labels.get(str(cls)) and len(class_img_labels[str(cls)]) >= 2]
    # Negatives require at least 1 image per class.
    valid_negative_classes = [cls for cls in class_list if class_img_labels.get(str(cls)) and len(class_img_labels[str(cls)]) >= N-2]

    # 2. Check if it's even possible to form an N-tuple with the available valid classes.
    # We need at least 1 anchor class and N-2 negative classes.
    if len(valid_negative_classes) < N - 1:
        print("Warning: Not enough classes with images to form N-tuples. Returning empty tensors.")
        return torch.empty(0), torch.empty(0), torch.empty(0)
    # --- End of Fix ---

    for cls in valid_anchor_classes:  #<-- Using 'cls' as requested.
        class_imgs = class_img_labels[str(cls)]

        # Get a list of all possible negative classes that are valid for this anchor.
        neg_classes = [c for c in valid_negative_classes if c != cls] #<-- Using 'neg_classes'.

        # Check if there are enough negative classes for this specific anchor.
        if len(neg_classes) < N - 2:
            continue

        for i in range(min(samples_per_class, len(class_imgs))):
            anchor = class_imgs[i]
            # pick a different positive from the same class
            pos_idx = choice([j for j in range(len(class_imgs)) if j != i])
            positive = class_imgs[pos_idx]

            negative_samples = []

            # --- Start of Fix ---
            # Use a while loop structure as in the original, but sample from the
            # pre-validated `neg_classes` list to ensure unique negative classes.
            neg_classes_to_sample_from = neg_classes.copy()
            while len(negative_samples) < (N - 2):
                neg_cls = choice(neg_classes_to_sample_from) #<-- Using 'neg_cls'. Safe because list is pre-validated.
                neg_classes_to_sample_from.remove(neg_cls) # Ensures we don't pick the same class twice.

                neg_imgs = class_img_labels[str(neg_cls)] #<-- Using 'neg_imgs'.
                neg_img = choice(neg_imgs) #<-- Using 'neg_img'.
                negative_samples.append(neg_img)
            # --- End of Fix ---

            anchors.append(anchor)
            positives.append(positive)
            negatives.append(torch.stack(negative_samples))

    if not anchors:
        print("Warning: No valid N-tuples could be generated.")
        return torch.empty(0), torch.empty(0), torch.empty(0)

    anchors = torch.stack(anchors)
    positives = torch.stack(positives)
    negatives = torch.stack(negatives)

    return anchors, positives, negatives


In [11]:
#@title Example test case for ntuple_reid_data function
import torch
import numpy as np

# Simulated dataset: 3 classes, each with 3 small 2x2 images
class_img_labels = {
    '0': [torch.tensor([[0, 0], [0, 0]]), torch.tensor([[0, 1], [0, 1]]), torch.tensor([[0, 2], [0, 2]]), torch.tensor([[0, 3], [0, 3]]), torch.tensor([[0, 4], [0, 4]]), torch.tensor([[0, 5], [0, 5]]), torch.tensor([[0, 6], [0, 6]]), torch.tensor([[0, 7], [0, 7]]), torch.tensor([[0, 8], [0, 8]])],
    '1': [torch.tensor([[1, 0], [1, 0]]), torch.tensor([[1, 1], [1, 1]]), torch.tensor([[1, 2], [1, 2]]), torch.tensor([[1, 3], [1, 3]]), torch.tensor([[1, 4], [1, 4]]), torch.tensor([[1, 5], [1, 5]]), torch.tensor([[1, 6], [1, 6]]), torch.tensor([[1, 7], [1, 7]]), torch.tensor([[1, 8], [1, 8]])],
    '2': [torch.tensor([[2, 0], [2, 0]]), torch.tensor([[2, 1], [2, 1]]), torch.tensor([[2, 2], [2, 2]]), torch.tensor([[2, 3], [2, 3]]), torch.tensor([[2, 4], [2, 4]]), torch.tensor([[2, 5], [2, 5]]), torch.tensor([[2, 6], [2, 6]]), torch.tensor([[2, 7], [2, 7]]), torch.tensor([[2, 8], [2, 8]])],
    '3': [torch.tensor([[3, 0], [3, 0]]), torch.tensor([[3, 1], [3, 1]]), torch.tensor([[3, 2], [3, 2]]), torch.tensor([[3, 3], [3, 3]]), torch.tensor([[3, 4], [3, 4]]), torch.tensor([[3, 5], [3, 5]]), torch.tensor([[3, 6], [3, 6]]), torch.tensor([[3, 7], [3, 7]]), torch.tensor([[3, 8], [3, 8]])],
    '4': [torch.tensor([[4, 0], [4, 0]]), torch.tensor([[4, 1], [4, 1]]), torch.tensor([[4, 2], [4, 2]]), torch.tensor([[4, 3], [4, 3]]), torch.tensor([[4, 4], [4, 4]]), torch.tensor([[4, 5], [4, 5]]), torch.tensor([[4, 6], [4, 6]]), torch.tensor([[4, 7], [4, 7]]), torch.tensor([[4, 8], [4, 8]])],
    '5': [torch.tensor([[5, 0], [5, 0]]), torch.tensor([[5, 1], [5, 1]]), torch.tensor([[5, 2], [5, 2]]), torch.tensor([[5, 3], [5, 3]]), torch.tensor([[5, 4], [5, 4]]), torch.tensor([[5, 5], [5, 5]]), torch.tensor([[5, 6], [5, 6]]), torch.tensor([[5, 7], [5, 7]]), torch.tensor([[5, 8], [5, 8]])],
    '6': [torch.tensor([[6, 0], [6, 0]]), torch.tensor([[6, 1], [6, 1]]), torch.tensor([[6, 2], [6, 2]]), torch.tensor([[6, 3], [6, 3]]), torch.tensor([[6, 4], [6, 4]]), torch.tensor([[6, 5], [6, 5]]), torch.tensor([[6, 6], [6, 6]]), torch.tensor([[6, 7], [6, 7]]), torch.tensor([[6, 8], [6, 8]])],
    '7': [torch.tensor([[7, 0], [7, 0]]), torch.tensor([[7, 1], [7, 1]]), torch.tensor([[7, 2], [7, 2]]), torch.tensor([[7, 3], [7, 3]]), torch.tensor([[7, 4], [7, 4]]), torch.tensor([[7, 5], [7, 5]]), torch.tensor([[7, 6], [7, 6]]), torch.tensor([[7, 7], [7, 7]]), torch.tensor([[7, 8], [7, 8]])],
    '8': [torch.tensor([[8, 0], [8, 0]]), torch.tensor([[8, 1], [8, 1]]), torch.tensor([[8, 2], [8, 2]]), torch.tensor([[8, 3], [8, 3]]), torch.tensor([[8, 4], [8, 4]]), torch.tensor([[8, 5], [8, 5]]), torch.tensor([[8, 6], [8, 6]]), torch.tensor([[8, 7], [8, 7]]), torch.tensor([[8, 8], [8, 8]])],
    '9': [torch.tensor([[9, 0], [9, 0]]), torch.tensor([[9, 1], [9, 1]]), torch.tensor([[9, 2], [9, 2]]), torch.tensor([[9, 3], [9, 3]]), torch.tensor([[9, 4], [9, 4]]), torch.tensor([[9, 5], [9, 5]]), torch.tensor([[9, 6], [9, 6]]), torch.tensor([[9, 7], [9, 7]]), torch.tensor([[9, 8], [9, 8]])],
}

class_list = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

# Assume your ntuple_reid_data function is defined
anchors, positives, negatives = ntuple_reid_data(class_img_labels, class_list, N=5, samples_per_class=2)

print("Anchor:")
print(anchors.shape)
print(anchors[0])

print("\nPositive:")
print(positives.shape)
print(positives[0])

print("\nNegatives: ")
print(negatives.shape)
print(negatives[0])


Anchor:
torch.Size([20, 2, 2])
tensor([[0, 0],
        [0, 0]])

Positive:
torch.Size([20, 2, 2])
tensor([[0, 8],
        [0, 8]])

Negatives: 
torch.Size([20, 3, 2, 2])
tensor([[[1, 6],
         [1, 6]],

        [[6, 1],
         [6, 1]],

        [[9, 7],
         [9, 7]]])


In [12]:
def pair_pretrain_on_dataset(source, project_path='./', dataset_parent='./',val_perc=0.5):

  if source == 'market':
      train_list = project_path + source+ '/train.txt'
      train_dir = dataset_parent + source+ '/bounding_box_train'
      class_count = 750
      test_list = project_path + source+ '/test.txt'
      test_dir = dataset_parent + source+ '/bounding_box_test'

  elif source == 'cuhk03':
      train_list = os.path.join(data_dir, 'train.txt')
      train_dir = os.path.join(data_dir, 'images_labeled')
      class_count = None

      test_list = os.path.join(data_dir, 'test.txt')
      test_dir = os.path.join(data_dir, 'images_labeled')

  else:
      train_list = 'unknown'
      train_dir = 'unknown'
      class_count = -1

  class_img_labels = reid_data_prepare(train_list, train_dir)
  class_train = class_img_labels
  class_num = len(class_img_labels)

  if val_perc > 0: # set val data percentage
    class_val = sample(list(np.arange(len(class_img_labels))), int(len(class_img_labels)*val_perc))
    class_train = list(set(np.arange(len(class_img_labels))) - set(class_val))

    train =ntuple_reid_data(class_img_labels, class_train)
    print("loaded train data")
    val = ntuple_reid_data(class_img_labels, class_val)
    print("loaded validation data")

    class_test_dict = reid_data_prepare(test_list, test_dir)
    class_test = np.arange(len(class_test_dict))

    test = ntuple_reid_data(class_test_dict, class_test, train=False)

    if val:
        print("len train class:", len(train[1]),"len val class:", len(val[1]), "len test class:", len(test[1]))
    else:
        print("len train class:", len(train[1]),"len val class:", 0, "len test class:", len(test[1]))

    return train, val, test,class_img_labels, class_val,class_num

In [13]:
import torch
from torch.utils.data import Dataset

class NTupleDataset(Dataset):
    """
    A Dataset class for pre-computed N-tuples.
    Assumes that anchors, positives, and negatives have already been sampled.
    """

    def __init__(self, anchors, positives, negatives):
        """
        Args:
            anchors (list or Tensor): A list/tensor of all anchor images.
            positives (list or Tensor): A list/tensor of all positive images.
            negatives (list or Tensor): A list/tensor of all negative image sets.
        """
        # Ensure all lists have the same length
        assert len(anchors) == len(positives) == len(negatives), \
            "All data lists must have the same length."

        self.anchors = anchors
        self.positives = positives
        self.negatives = negatives
        self.data_len = len(anchors)

    def __len__(self):
        return self.data_len

    def __getitem__(self, index):
        """
        Returns the pre-computed N-tuple at a given index.
        """
        anchor = self.anchors[index]
        positive = self.positives[index]
        negative_set = self.negatives[index]

        return anchor, positive, negative_set

In [14]:
import torch
from torch.utils.data import DataLoader, ConcatDataset

def loadbatches(train, val, test, loader_kargs, batch_size):
    """
    Function to load the batches for the dataset.
    This version works with any standard PyTorch Dataset object, including PrecomputedNTupleDataset.

    Parameters
    ----------
    train : torch.utils.data.Dataset
        Train dataset object (e.g., an instance of PrecomputedNTupleDataset).
    val : torch.utils.data.Dataset
        Validation dataset object.
    test : torch.utils.data.Dataset
        Test dataset object.
    loader_kargs : dict
        Loader arguments (e.g., num_workers, pin_memory).
    batch_size : int
        The size of the batch.
    """

    # Use the standard len() function which works with PyTorch Datasets
    ntrain = len(train)
    ntest = len(test)
    print(f"Train data length: {ntrain}, Test data length: {ntest}")

    # Initialize all loaders to None
    train_loader, prior_loader, set_bound_1batch, set_val_bound = None, None, None, None

    if val:
        concat_data = ConcatDataset([train, val])

        # Main loader for training on both train and validation sets
        train_loader = DataLoader(concat_data, batch_size=batch_size, shuffle=True, **loader_kargs)
        # Loader for the validation/prior set
        prior_loader = DataLoader(val, batch_size=batch_size, **loader_kargs)
        # Single-batch loader for the train set
        set_bound_1batch = DataLoader(train, batch_size=ntrain, **loader_kargs)
        # Standard-batch loader for the train set (for validation-like calculations)
        set_val_bound = DataLoader(train, batch_size=batch_size)
    else:
        # If no validation set, the train_loader only uses the training data
        train_loader = DataLoader(train, batch_size=batch_size, shuffle=True, **loader_kargs)


    # Standard and single-batch loaders for the test set
    test_1batch = DataLoader(test, batch_size=ntest, **loader_kargs)
    test_loader = DataLoader(test, batch_size=batch_size, **loader_kargs)


    return train_loader, test_loader, prior_loader, set_bound_1batch, test_1batch, set_val_bound

In [15]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class MetaLearner(nn.Module):
    """
    Implements the meta-learner subnet phi(·) from Equation (7) of the paper.
    It takes instance features and maps them to refined reference nodes.

    Args:
        embedding_dim (int): The dimension of the input feature embeddings (d).
        reduction_ratio (int): The ratio for dimension reduction in the bottleneck layer.
    """
    def __init__(self, embedding_dim=1024, reduction_ratio=8):
        super(MetaLearner, self).__init__()
        bottleneck_dim = embedding_dim // reduction_ratio

        self.mapper = nn.Sequential(
            nn.Linear(embedding_dim, bottleneck_dim),
            nn.BatchNorm1d(bottleneck_dim),
            # The paper does not specify an activation, but ReLU is a common choice.
            # nn.ReLU(inplace=True),
            nn.Linear(bottleneck_dim, embedding_dim)
        )

    def forward(self, x):
        return self.mapper(x)


class NTupleLoss(nn.Module):
    """
    Implementation of N-tuple and Meta Prototypical N-tuple (MPN-tuple) loss.

    Args:
        mode (str): The loss mode. Must be one of 'regular' or 'mpn'.
                    - 'regular': Standard N-tuple loss using instance features directly.
                    - 'mpn': Meta Prototypical N-tuple loss using a meta-learner.
        embedding_dim (int): The dimension of the feature embeddings.
                             Required only if mode is 'mpn'.
        initial_temp (float): The initial temperature (tau) for scaling similarities.
    """
    def __init__(self, mode='mpn', embedding_dim=1024, initial_temp=0.05):
        super(NTupleLoss, self).__init__()

        if mode not in ['regular', 'mpn']:
            raise ValueError("Mode must be either 'regular' or 'mpn'")
        self.mode = mode

        # The paper makes the temperature a learnable parameter by learning s = 1/tau
        # We will do the same for flexibility.
        self.log_s = nn.Parameter(torch.log(torch.tensor(1.0 / initial_temp)))

        if self.mode == 'mpn':
            self.meta_learner = MetaLearner(embedding_dim=embedding_dim)

    def forward(self, anchor_embed, positive_embed, negative_embeds):
        """
        Calculates the N-tuple loss.

        Args:
            anchor_embed (torch.Tensor): Embeddings of the anchor samples.
                                         Shape: (batch_size, embedding_dim)
            positive_embed (torch.Tensor): Embeddings of the positive samples.
                                          Shape: (batch_size, embedding_dim)
            negative_embeds (torch.Tensor): Embeddings of the negative samples.
                                           Shape: (batch_size, N-2, embedding_dim)

        Returns:
            torch.Tensor: The calculated N-tuple loss for the batch.
        """
        # Get the reference nodes for positive and negative samples
        if self.mode == 'mpn':
            # For MPN loss, pass positives and negatives through the meta-learner
            # to get the reference nodes (prototypes).
            # The paper averages multiple instances for a prototype; here we assume
            # the provided single positive/negative is the basis for its prototype.
            positive_ref = self.meta_learner(positive_embed)

            # Reshape negatives to pass through the linear layers of the meta-learner
            batch_size, n_negatives, embed_dim = negative_embeds.shape
            negatives_flat = negative_embeds.view(-1, embed_dim)
            negative_ref_flat = self.meta_learner(negatives_flat)
            negative_ref = negative_ref_flat.view(batch_size, n_negatives, embed_dim)

        else: # 'regular' mode
            # For regular N-tuple loss, the instance embeddings are the reference nodes.
            positive_ref = positive_embed
            negative_ref = negative_embeds

        # --- Calculate similarities ---
        # Cosine similarity is used as per the paper
        sim_positive = F.cosine_similarity(anchor_embed, positive_ref)

        # To calculate similarity between anchor and all negatives, we need to unsqueeze
        # the anchor to enable broadcasting across the N-2 dimension.
        # anchor_embed shape: (B, D) -> (B, 1, D)
        # negative_ref shape: (B, N-2, D)
        sim_negatives = F.cosine_similarity(anchor_embed.unsqueeze(1), negative_ref, dim=2)

        # --- Formulate as a classification problem ---
        # The goal is to classify the anchor as belonging to the positive reference
        # over all negative references. This can be solved with CrossEntropyLoss.

        # The logits are the scaled similarities.
        # Concatenate the positive similarity with all negative similarities.
        # Shape: (B, 1+ (N-2)) -> (B, N-1)
        logits = torch.cat([sim_positive.unsqueeze(1), sim_negatives], dim=1)

        # Scale logits by the learned temperature parameter s = 1/tau
        logits *= torch.exp(self.log_s)

        # The target label for every sample is 0, because the positive class
        # is always at index 0 of our logits tensor.
        targets = torch.zeros(logits.size(0), dtype=torch.long, device=logits.device)

        # Calculate the cross-entropy loss, which is equivalent to the
        loss = F.cross_entropy(logits, targets)

        return loss


In [16]:
#@title Example test case for ntuple_loss function
from loss import NTupleLoss

BATCH_SIZE = 16
EMBEDDING_DIM = 1024
N_NEGATIVES = 20 # This means N = 16 (1 anchor + 1 positive + 14 negatives)

# --- Dummy Data ---
anchor_features = torch.randn(BATCH_SIZE, EMBEDDING_DIM)
positive_features = torch.randn(BATCH_SIZE, EMBEDDING_DIM)
negative_features = torch.randn(BATCH_SIZE, N_NEGATIVES, EMBEDDING_DIM)

    # --- Instantiate the Loss Function ---

    # 1. MPN-tuple Loss
print("Testing MPN-tuple Loss:")
mpn_loss_fn = NTupleLoss(mode='mpn', embedding_dim=EMBEDDING_DIM)
loss_value_mpn = mpn_loss_fn(anchor_features, positive_features, negative_features)
print(f"  Calculated MPN Loss: {loss_value_mpn.item():.4f}")
print(f"  Learnable temperature param (s=1/tau): {torch.exp(mpn_loss_fn.log_s).item():.4f}")


    # 2. Regular N-tuple Loss
print("\nTesting Regular N-tuple Loss:")
regular_loss_fn = NTupleLoss(mode='regular')
loss_value_regular = regular_loss_fn(anchor_features, positive_features, negative_features)
print(f"  Calculated Regular N-tuple Loss: {loss_value_regular.item():.4f}")

Testing MPN-tuple Loss:
  Calculated MPN Loss: 3.1834
  Learnable temperature param (s=1/tau): 20.0000

Testing Regular N-tuple Loss:
  Calculated Regular N-tuple Loss: 3.3922


In [ ]:
# Import all necessary modules from your project
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset
from tqdm.notebook import trange, tqdm
import math
import numpy as np
import random

import os
os.environ['PYTORCH_ENABLE_MPS_FALLBACK'] = '1'

# --- Assume these are your adapted N-tuple components ---
# These imports assume your files are in a directory added to sys.path
# from data_utils import reid_data_prepare, loadbatches
# FIXED: Import the specific block class needed by ProbResNet_BN
from models import ResNet, ProbResNet_BN, ProbResidualBlock_bn
from bounds import PBBobj_Ntuple
from loss import NTupleLoss

# ==============================================================================
# NEW: Dynamic Sampling Dataset to prevent memory crashes
# ==============================================================================
class DynamicNTupleDataset(Dataset):
    """
    Dataset that samples N-tuples dynamically to save memory.
    It creates tuples on-the-fly in the __getitem__ method.
    """
    def __init__(self, class_img_labels, class_ids, N=4, samples_per_epoch_multiplier=4):
        self.class_img_labels = class_img_labels
        self.class_ids = class_ids  # The list of class IDs for this split (e.g., train_ids)
        self.N = N
        self.samples_per_epoch_multiplier = samples_per_epoch_multiplier

        # Create a flat list of all (image_tensor, class_id) pairs for this dataset split.
        # These are the potential anchors. We only include images from classes that
        # have at least 2 images, so a positive pair can always be formed.
        self.anchor_pool = []
        for cid in self.class_ids:
            cid_str = str(cid)
            if cid_str in self.class_img_labels and len(self.class_img_labels[cid_str]) >= 2:
                for img in self.class_img_labels[cid_str]:
                    self.anchor_pool.append({'img': img, 'cid': cid})
        
        if not self.anchor_pool:
            raise ValueError("No classes with enough images to form anchor-positive pairs.")

        # The length is the number of available anchors multiplied by a factor to control epoch size.
        self.length = len(self.anchor_pool) * self.samples_per_epoch_multiplier

    def __len__(self):
        return self.length

    def __getitem__(self, index):
        # Use modulo to cycle through the anchor pool
        anchor_info = self.anchor_pool[index % len(self.anchor_pool)]
        anchor_img = anchor_info['img']
        anchor_cid = anchor_info['cid']

        # 1. Sample a positive image from the same class
        positive_options = self.class_img_labels[str(anchor_cid)]
        positive_img = random.choice(positive_options)
        # Ensure the positive isn't the exact same tensor instance as the anchor
        if len(positive_options) > 1:
            while torch.equal(anchor_img, positive_img):
                positive_img = random.choice(positive_options)

        # 2. Sample N-2 negatives from different classes
        # Get a list of all possible negative class IDs that have at least one image
        possible_neg_cids = [cid for cid in self.class_ids if cid != anchor_cid and str(cid) in self.class_img_labels and self.class_img_labels[str(cid)]]
        
        # Check if enough unique negative classes are available
        if len(possible_neg_cids) < self.N - 2:
            # Fallback: if not enough unique classes, sample with replacement.
            # This is unlikely in a large dataset but makes the code robust.
            neg_cids_sample = random.choices(possible_neg_cids, k=self.N - 2)
        else:
            neg_cids_sample = random.sample(possible_neg_cids, self.N - 2)
            
        negative_imgs = [random.choice(self.class_img_labels[str(c)]) for c in neg_cids_sample]
        negatives_tensor = torch.stack(negative_imgs)
        
        return anchor_img, positive_img, negatives_tensor

# ==============================================================================
# Adapted ResNet (no changes from before)
# ==============================================================================
class ResNet(nn.Module):
    def __init__(self, num_classes=751):
        super(ResNet, self).__init__()
        resnet18_model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)
        self.resnet18_model = nn.Sequential(*list(resnet18_model.children())[:-1])
        self.fc1 = nn.Linear(512, num_classes)

    def forward(self, x):
        features = self.resnet18_model(x)
        embeddings = features.view(features.size(0), -1)
        return embeddings

# ==============================================================================
# MAIN EXPERIMENT RUNNER
# ==============================================================================
def run_ntuple_experiment(config):
    """
    Experiment runner adapted to use memory-efficient dynamic sampling.
    """
    # --- 1. SETUP ---
    print("--- Starting Experiment ---")
    print(f"Config: {config}")
    device = config['device']
    torch.manual_seed(7)
    np.random.seed(0)

    loader_kargs = {'num_workers': 0, 'pin_memory': True} if 'cuda' in device else {}
    rho_prior = math.log(math.exp(config['sigma_prior']) - 1.0)

    # --- 2. DATA PREPARATION ---
    print("\n--- Preparing Data ---")
    class_img_labels = reid_data_prepare(config['data_list_path'], config['data_dir_path'])
    all_class_ids = list(class_img_labels.keys())

    val_size = int(len(all_class_ids) * config['val_perc'])
    train_ids = all_class_ids[val_size:]
    val_ids = all_class_ids[:val_size]

    # --- MODIFIED: Use DynamicNTupleDataset instead of pre-computing ---
    print("Initializing dynamic datasets...")
    train_dataset = DynamicNTupleDataset(class_img_labels, train_ids, N=config['N'], samples_per_epoch_multiplier=config['samples_per_class'])
    val_dataset = DynamicNTupleDataset(class_img_labels, val_ids, N=config['N'], samples_per_epoch_multiplier=config['samples_per_class'])
    test_dataset = val_dataset # Using val set for testing as a placeholder
    # --------------------------------------------------------------------

    train_loader, test_loader, prior_loader, _, _, _ = loadbatches(
        train_dataset, val_dataset, test_dataset, loader_kargs, config['batch_size']
    )
    print("Data preparation complete.")

    # --- 3. MODEL INITIALIZATION ---
    print("\n--- Initializing Models ---")
    net0 = ResNet().to(device)
    
    # FIXED: Pass the ProbResidualBlock_bn class as the first argument
    net = ProbResNet_BN(ProbResidualBlock_bn, rho_prior=rho_prior, init_net=net0, device=device).to(device)
    
    optimizer = optim.SGD(net.parameters(), lr=config['learning_rate'], momentum=config['momentum'])

    # --- 4. SETUP FOR PAC-BAYES ---
    print("\n--- Setting up PAC-Bayes Objective ---")
    pbobj = PBBobj_Ntuple(
        objective=config['objective'],
        delta=config['delta'],
        delta_test=config['delta_test'],
        mc_samples=config['mc_samples'],
        kl_penalty=config['kl_penalty'],
        device=device,
        n_posterior=len(train_dataset),
        n_bound=len(val_dataset) if val_dataset else 0
    )
    ntuple_loss_fn = NTupleLoss(mode=config['ntuple_mode'], embedding_dim=512).to(device)

    # --- 5. MAIN TRAINING LOOP ---
    print("\n--- Starting Training ---")
    results = {}
    for epoch in trange(config['train_epochs'], desc="Training Progress"):
        net.train()
        for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}", leave=False):
            optimizer.zero_grad()
            bound, _, _ = pbobj.train_obj_ntuple(net, batch, ntuple_loss_fn)
            bound.backward()
            optimizer.step()

        # --- 6. PERIODIC EVALUATION ---
        if (epoch + 1) % config['test_interval'] == 0:
            if prior_loader:
                print(f"\n--- Evaluating at Epoch {epoch+1} ---")
                final_risk, kl, emp_risk, pseudo_acc = pbobj.compute_final_stats_risk(net, prior_loader, ntuple_loss_fn)
                print(f"  Certified N-Tuple Risk: {final_risk:.5f}")
                print(f"  KL Divergence: {kl:.5f}")
                print(f"  Empirical N-Tuple Risk (on val set): {emp_risk:.5f}")
                print(f"  Pseudo-Accuracy (on val set): {pseudo_acc:.4f}")
                results[epoch+1] = {'risk': final_risk, 'kl': kl, 'empirical_risk': emp_risk, 'pseudo_accuracy': pseudo_acc}

    print("\n--- Training Finished ---")
    return results


# --- Example Usage in a Notebook Cell ---
# Define your configuration
config = {
    'device': 'cuda',
    'data_list_path': '/Users/misanmeggison/Self-certified-Tuple-wise/cuhk03/train.txt', # From your notebook's data prep
    'data_dir_path': '/Users/misanmeggison/Self-certified-Tuple-wise/cuhk031/images_detected/', # From your notebook's data prep
    'val_perc': 0.2,
    'batch_size': 64,
    'learning_rate': 0.01,
    'momentum': 0.9,
    'sigma_prior': 0.1,
    'train_epochs': 5,
    'test_interval': 5, # Evaluate every 5 epochs
    'objective': 'fclassic',
    'delta': 0.025,
    'delta_test': 0.01,
    'mc_samples': 100,
    'kl_penalty': 1.0,
    'N': 4, # Number of elements in the tuple
    'samples_per_class': 4, # How many tuples to generate per person ID
    'ntuple_mode': 'mpn' # Use 'mpn' or 'regular'
}

In [18]:
# Run the experiment
experiment_results = run_ntuple_experiment(config)

--- Starting Experiment ---
Config: {'device': 'mps', 'data_list_path': '/Users/misanmeggison/Self-certified-Tuple-wise/cuhk03/train.txt', 'data_dir_path': '/Users/misanmeggison/Self-certified-Tuple-wise/cuhk031/images_detected/', 'val_perc': 0.2, 'batch_size': 64, 'learning_rate': 0.01, 'momentum': 0.9, 'sigma_prior': 0.1, 'train_epochs': 5, 'test_interval': 5, 'objective': 'fclassic', 'delta': 0.025, 'delta_test': 0.01, 'mc_samples': 100, 'kl_penalty': 1.0, 'N': 4, 'samples_per_class': 4, 'ntuple_mode': 'mpn'}

--- Preparing Data ---
Loaded and transformed image: /Users/misanmeggison/Self-certified-Tuple-wise/cuhk031/images_detected/1_001_1_01.png
Loaded and transformed image: /Users/misanmeggison/Self-certified-Tuple-wise/cuhk031/images_detected/1_001_1_02.png
Loaded and transformed image: /Users/misanmeggison/Self-certified-Tuple-wise/cuhk031/images_detected/1_001_1_03.png
Loaded and transformed image: /Users/misanmeggison/Self-certified-Tuple-wise/cuhk031/images_detected/1_001_1_0

Using cache found in /Users/misanmeggison/.cache/torch/hub/pytorch_vision_v0.10.0
/Users/misanmeggison/Downloads/cukh03/.venv/lib/python3.13/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/misanmeggison/Downloads/cukh03/.venv/lib/python3.13/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)



--- Setting up PAC-Bayes Objective ---

--- Starting Training ---


Training Progress:   0%|          | 0/5 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/461 [00:00<?, ?it/s]

Sampled epsilon: torch.float32/mps:0
Sigma dtype/device: torch.float32/mps:0
Input dtype/device: torch.float32/mps:0
Weight dtype/device: torch.float32/mps:0
Sampled epsilon: torch.float32/mps:0
Sigma dtype/device: torch.float32/mps:0
Sampled epsilon: torch.float32/mps:0
Sigma dtype/device: torch.float32/mps:0
Sampled epsilon: torch.float32/mps:0
Sigma dtype/device: torch.float32/mps:0
Input dtype/device: torch.float32/mps:0
Weight dtype/device: torch.float32/mps:0
Sampled epsilon: torch.float32/mps:0
Sigma dtype/device: torch.float32/mps:0
Sampled epsilon: torch.float32/mps:0
Sigma dtype/device: torch.float32/mps:0
Sampled epsilon: torch.float32/mps:0
Sigma dtype/device: torch.float32/mps:0
Input dtype/device: torch.float32/mps:0
Weight dtype/device: torch.float32/mps:0
Sampled epsilon: torch.float32/mps:0
Sigma dtype/device: torch.float32/mps:0
Sampled epsilon: torch.float32/mps:0
Sigma dtype/device: torch.float32/mps:0
Sampled epsilon: torch.float32/mps:0
Sigma dtype/device: torch.f

Error: command buffer exited with error status.
	The Metal Performance Shaders operations encoded on it may not have completed.
	Error: 
	(null)
	Insufficient Memory (00000008:kIOGPUCommandBufferCallbackErrorOutOfMemory)
	<AGXG13GFamilyCommandBuffer: 0x4daba9a70>
    label = <none> 
    device = <AGXG13GDevice: 0x10cc28600>
        name = Apple M1 
    commandQueue = <AGXG13GFamilyCommandQueue: 0x10cc07200>
        label = <none> 
        device = <AGXG13GDevice: 0x10cc28600>
            name = Apple M1 
    retainedReferences = 1


Sampled epsilon: torch.float32/mps:0
Sigma dtype/device: torch.float32/mps:0
Sampled epsilon: torch.float32/mps:0
Sigma dtype/device: torch.float32/mps:0
Sampled epsilon: torch.float32/mps:0
Sigma dtype/device: torch.float32/mps:0
Input dtype/device: torch.float32/mps:0
Weight dtype/device: torch.float32/mps:0
Sampled epsilon: torch.float32/mps:0
Sigma dtype/device: torch.float32/mps:0
Sampled epsilon: torch.float32/mps:0
Sigma dtype/device: torch.float32/mps:0
Sampled epsilon: torch.float32/mps:0
Sigma dtype/device: torch.float32/mps:0
Input dtype/device: torch.float32/mps:0
Weight dtype/device: torch.float32/mps:0
Sampled epsilon: torch.float32/mps:0
Sigma dtype/device: torch.float32/mps:0
Sampled epsilon: torch.float32/mps:0
Sigma dtype/device: torch.float32/mps:0
Sampled epsilon: torch.float32/mps:0
Sigma dtype/device: torch.float32/mps:0
Input dtype/device: torch.float32/mps:0
Weight dtype/device: torch.float32/mps:0
Sampled epsilon: torch.float32/mps:0
Sigma dtype/device: torch.f

RuntimeError: MPS backend out of memory (MPS allocated: 8.13 GiB, other allocations: 362.50 MiB, max allowed: 9.07 GiB). Tried to allocate 784.00 MiB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).

In [ ]:
!pip3 install --pre torch torchvision torchaudio --index-url https://download.pytorch.org/whl/nightly/cpu

Looking in indexes: https://download.pytorch.org/whl/nightly/cpu
INFO: pip is looking at multiple versions of torchaudio to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of torchaudio to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 MB 19.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 17.6 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.7.1
    Uninstalling torch-2.7.1:━━━━━━━━━━━━━━━━━━━ 0/3 [torch]
      Successfu

In [ ]:
import torch

try:
    # First, check that the cuda device is available
    if torch.cuda.is_available():
        cuda_device = torch.device("cuda")
        print("CUDA device found.")
        
        # Now, try to create a tensor on the CUDA device
        print("Attempting to create a tensor on CUDA device...")
        x = torch.randn(2, 2, device=cuda_device)
        print("Successfully created a tensor on the CUDA device:")
        print(x)
    else:
        print("CUDA device not found.")
except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:
# --- DEBUG CELL ---
# Purpose: To test if the model's forward pass fails in isolation.

print("--- Starting Minimal Forward Pass Test ---")
try:
    # 1. Get the device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    # 2. Initialize your model directly
    # Make sure you have imported ProbResNet_BN from models.py
    net = ProbResNet_BN(
        metric_embedding_dim=128,  # Using an example value
        n_classes=10,            # Using an example value
        device=device,
        sigma_prior=0.1          # Using an example value
    ).to(device)

    # 3. Set the model to training mode. This is CRITICAL.
    # The error happens during training because your `ResProbConv2d.forward` method
    # uses `if self.training or sample:` to decide whether to sample new weights.
    net.train()
    print("Model initialized on CUDA and set to training mode.")

    # 4. Create a dummy input tensor with the correct shape and device.
    # Your dataset seems to use Grayscale and resizes images to 160x60.
    # The shape is (N, C, H, W) -> (batch_size, 1, 160, 60)
    dummy_input = torch.randn(4, 1, 160, 60, device=device)
    print(f"Dummy input created on device: {dummy_input.device}")

    # 5. Call the model's forward pass
    print("Attempting model forward pass...")
    output = net(dummy_input)
    print("--- Minimal Forward Pass Test SUCCEEDED ---")
    print(f"Output tensor device: {output.device}")
    print(f"Output shape: {output.shape}")

except Exception as e:
    print(f"--- Minimal Forward Pass Test FAILED ---")
    import traceback
    traceback.print_exc()